In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
RANDOM_STATE = 42

In [3]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=10)

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [5]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
pred = model_lr.predict(X_test)
print('R2 LinearRegression', r2_score(y_test, pred)) 

R2 LinearRegression 0.670933983911564


In [6]:
model_ridge = Ridge()
model_ridge.fit(X_train, y_train)
pred = model_ridge.predict(X_test)
print('R2 Ridge',r2_score(y_test, pred)) 

R2 Ridge 0.6655627286231769


In [7]:
model_lasso = Lasso()
model_lasso.fit(X_train, y_train)
pred = model_lasso.predict(X_test)
print('R2 model_lasso',r2_score(y_test, pred)) 

R2 model_lasso 0.5350073630794527


3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [8]:
best_model_lasso = None
best_model_Ridge = None
best_result_lasso = 0
best_result_Ridge = 0
params_grid=[]
for n in range(-5,5,1):
    params_grid.append(10**n)
model_Ridge = Ridge()
model_lasso = Lasso()
grid_Ridge = GridSearchCV(model_Ridge, {'alpha': params_grid})
grid_lasso = GridSearchCV(model_lasso, {'alpha': params_grid})
grid_Ridge.fit(X_train, y_train)
grid_lasso.fit(X_train, y_train)
result_lasso = grid_lasso.score(X_test, y_test)
result_Ridge = grid_Ridge.score(X_test, y_test)
if result_lasso > best_result_lasso:
    best_model_lasso = grid_lasso 
    best_result_lasso = result_lasso
if result_Ridge > best_result_Ridge:
    best_model_Ridge = grid_Ridge 
    best_result_Ridge = result_Ridge
print(f'Лучший результат  (LassoCV): {best_result_lasso:.3f}')
print(f'Лучший результат  (RidgeCV): {best_result_Ridge:.3f}')

Лучший результат  (LassoCV): 0.671
Лучший результат  (RidgeCV): 0.670


In [9]:
from sklearn.linear_model import RidgeCV, LassoCV
alphas = 10**np.linspace(-5,5,11)
model_Ridge_CV = RidgeCV(alphas=alphas,scoring='r2').fit(X_train, y_train)
pred_ridge = model_Ridge_CV.predict(X_test)


model_Lasso_CV = LassoCV(alphas=alphas).fit(X_train, y_train)
pred_lasso = model_Lasso_CV.predict(X_test)

print(f"Лучший результат (RidgeCV): {r2_score(y_test, pred_ridge):.3f}")
print(f"Лучший результат (LassoCV): {r2_score(y_test, pred_lasso):.3f}")


Лучший результат (RidgeCV): 0.671
Лучший результат (LassoCV): 0.671


Результат при подборе коэффициента регуляризации улучшился у обоеих моделей, у Lasso результат улучшился значительнее.

4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

pipeline_SS = Pipeline([
    ('scaling', StandardScaler()),
    ('regression', Ridge())
])

pipeline_MMS = Pipeline([
    ('scaling', MinMaxScaler()),
    ('regression', Ridge())
])

model_Ridge_ss = pipeline_SS.fit(X_train, y_train)

model_Ridge_mms = pipeline_MMS.fit(X_train, y_train)

print('Ridge')
print(f"Лучший результат (StandardScaler): {model_Ridge_ss.score(X_test,y_test):.3f}")
print(f"Лучший результат (MinMaxScaler): {model_Ridge_mms.score(X_test,y_test):.3f}")

Ridge
Лучший результат (StandardScaler): 0.671
Лучший результат (MinMaxScaler): 0.666


In [11]:
pipeline_SS = Pipeline([
    ('scaling', StandardScaler()),
    ('regression', Lasso())
])

pipeline_MMS = Pipeline([
    ('scaling', MinMaxScaler()),
    ('regression', Lasso())
])

model_Lasso_ss = pipeline_SS.fit(X_train, y_train)

model_Lasso_mms = pipeline_MMS.fit(X_train, y_train)

print('Lasso')
print(f"Лучший результат (StandardScaler): {model_Lasso_ss.score(X_test,y_test):.3f}")
print(f"Лучший результат (MinMaxScaler): {model_Lasso_mms.score(X_test,y_test):.3f}")



Lasso
Лучший результат (StandardScaler): 0.608
Лучший результат (MinMaxScaler): 0.094


При использовании масштабирования значение r2 метрики не изменились для Ridge модели, для модели Lasso при MinMaxScaler результат сильно ухудшился.

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [12]:
pipe = Pipeline(steps=[
    ('scaler', 'passthrough'),
    ('clf', 'passthrough')
])
params = [{
    'scaler': [StandardScaler(), MinMaxScaler()],
    'clf': [Ridge()],
    'clf__alpha': alphas
}]
search_Ridge = GridSearchCV(pipe, params, scoring='r2', cv=10, n_jobs=-1).fit(X_train, y_train)

params = [{
    'scaler': [StandardScaler(), MinMaxScaler()],
    'clf': [Lasso()],
    'clf__alpha': alphas
}]
search_Lasso = GridSearchCV(pipe, params, scoring='r2', cv=10, n_jobs=-1).fit(X_train, y_train)

print(f"Ridge R2 : {search_Ridge.score(X_test,y_test):.3f}")
print(f"Lasso R2 : {search_Lasso.score(X_test,y_test):.3f}")

Ridge R2 : 0.672
Lasso R2 : 0.671


Подбор коэффициента регуляризации на масштабированных данных для Ridge и Lasso улучшения метрики R2 не дал.

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [13]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, interaction_only=True)

pipeline_SS = Pipeline([
    ('scaling', StandardScaler()),
    ('poly', poly),
    ('regression', Ridge())
])

pipeline_MMS = Pipeline([
    ('scaling', MinMaxScaler()),
    ('poly', poly),
    ('regression', Ridge())

])


Ridge_SS = pipeline_SS.fit(X_train, y_train)

Ridge_MMS = pipeline_MMS.fit(X_train, y_train)

print('Ridge')
print(f"R2 (StandardScaler): {Ridge_SS.score(X_test,y_test):.3f}")
print(f"R2 (MinMaxScaler): {Ridge_MMS.score(X_test,y_test):.3f}")


pipeline_SS = Pipeline([
    ('scaling', StandardScaler()),
    ('poly', poly),
    ('regression', Lasso())
])

pipeline_MMS = Pipeline([
    ('scaling', MinMaxScaler()),
    ('poly', poly),
    ('regression', Lasso())

])


Lasso_SS = pipeline_SS.fit(X_train, y_train)

Lasso_MMS = pipeline_MMS.fit(X_train, y_train)

print('Lasso')
print(f"R2 (StandardScaler): {Lasso_SS.score(X_test,y_test):.3f}")
print(f"R2 (MinMaxScaler): {Lasso_MMS.score(X_test,y_test):.3f}")



Ridge
R2 (StandardScaler): 0.879
R2 (MinMaxScaler): 0.783
Lasso
R2 (StandardScaler): 0.741
R2 (MinMaxScaler): 0.088


Для модели Ridge значение метрики R2 улучшилось для обоих вариантов масштабирования,
для модели Lasso при StandardScaler R2 улучшилась, при MinMaxScaler R2 незначительно ухудшилась.

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

In [14]:
%%time

pipe = Pipeline(steps=[
    ('scaler', 'passthrough'),
    ('poly', 'passthrough'),
    ('clf', 'passthrough')
])

params = [{
    'scaler': [StandardScaler(), MinMaxScaler()],
    'poly': [PolynomialFeatures(degree=2,interaction_only=True),
             PolynomialFeatures(degree=3,interaction_only=True),
             PolynomialFeatures(degree=2,interaction_only=False),
             PolynomialFeatures(degree=3,interaction_only=False)],
    'clf': [Ridge(), Lasso()],
    'clf__alpha': alphas
}]

search = GridSearchCV(pipe, params, scoring='r2', cv=10, n_jobs=-1).fit(X_train, y_train)


print(f'best model - {search.best_params_["clf"]}\n'
      f'Лучшее значение R2 - {search.score(X_test, y_test):.3f}\n'
      f'scaler - {search.best_params_["scaler"]}\n'
      f'poly - {search.best_params_["poly"]}')


best model - Lasso(alpha=0.0001)
Лучшее значение R2 - 0.867
scaler - MinMaxScaler()
poly - PolynomialFeatures(degree=3, interaction_only=True)
Wall time: 15.3 s


При подборе типа регуляризации (L1,L2), коэффициента регуляризации, метода масштабирования и степени полинома в PolynomialFeatures получилось достигнуть лучшее значение R2 = 0.867

http://archive.ics.uci.edu/ml/datasets/Adult

In [15]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [16]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [17]:
df = data.copy()
df.iloc[:,14] = df.iloc[:,14].replace(['<=50K','>50K'],[0,1])


In [18]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [19]:
df.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
dtype: int64

Нулевых значений нет.

10. Выберите колонки с числовыми и категориальными переменными.

In [20]:
cols = X.columns

X_num_cols = X._get_numeric_data().columns
X_cat_cols = list(set(cols) - set(X_num_cols))
X_num_cols = X_num_cols.to_list()
X_cat_cols

[1, 3, 5, 6, 7, 8, 9, 13]

In [21]:
X_num_cols

[0, 2, 4, 10, 11, 12]

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

In [22]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

categorical_transformer = OneHotEncoder(handle_unknown="ignore")

numeric_transformer = MinMaxScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, X_num_cols),
        ("cat", categorical_transformer, X_cat_cols),
    ]
)

clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)


12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [23]:
y.value_counts()

0    37155
1    11687
Name: 14, dtype: int64

In [24]:
print('Constant prediction')
print(f'Accuracy: {accuracy_score(y, np.array([0 for _ in range(len(y))]))}')
print(f'F1-score: {f1_score(y, np.array([0 for _ in range(len(y))]))}')


Constant prediction
Accuracy: 0.7607182343065395
F1-score: 0.0


13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

In [25]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC, LinearSVC

clf_lg = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

print('LogisticRegression')
print('accuracy', cross_val_score(clf_lg, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_lg, X, y, cv=5, scoring = 'f1').mean())

clf_lsvc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LinearSVC())]
)
print('LinearSVC')
print('accuracy', cross_val_score(clf_lsvc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_lsvc, X, y, cv=5, scoring = 'f1').mean())


clf_svc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", SVC())]
)
print('SVC')
print('accuracy', cross_val_score(clf_svc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_svc, X, y, cv=5, scoring = 'f1').mean())



LogisticRegression
accuracy 0.8511323258508231
f1 0.6564181874564248
LinearSVC
accuracy 0.8529135478362626
f1 0.6578163598233313
SVC
accuracy 0.8399739469040318
f1 0.6200828429738441


Результаты метрик улучшились на каждой модели.

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

In [26]:
(X == '?').sum()

0        0
1     2799
2        0
3        0
4        0
5        0
6     2809
7        0
8        0
9        0
10       0
11       0
12       0
13     857
dtype: int64

In [27]:
from sklearn.impute import SimpleImputer
data_columns = X.columns
imputer = SimpleImputer(missing_values='?', strategy='most_frequent')
new_data = pd.DataFrame(imputer.fit_transform(X), columns=data_columns)
new_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States
48838,64,Private,321403,HS-grad,9,Widowed,Prof-specialty,Other-relative,Black,Male,0,0,40,United-States
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States


15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

In [28]:
%%time
clf_lg = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

print('LogisticRegression')
print('accuracy', cross_val_score(clf_lg, new_data, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_lg, new_data, y, cv=5, scoring = 'f1').mean())



clf_lsvc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LinearSVC())]
)
print('LinearSVC')
print('accuracy', cross_val_score(clf_lsvc, new_data, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_lsvc, new_data, y, cv=5, scoring = 'f1').mean())


clf_svc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", SVC())]
)
print('SVC')
print('accuracy', cross_val_score(clf_svc, new_data, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_svc, new_data, y, cv=5, scoring = 'f1').mean())


LogisticRegression
accuracy 0.8507432753107848
f1 0.6543407046175806
LinearSVC
accuracy 0.8508251523375897
f1 0.650945261381221
SVC
accuracy 0.8395848963639935
f1 0.6169178164345149
Wall time: 12min 20s


Результаты метрик значительно не изменились.

16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

In [29]:
df_clear = df.replace('?', np.nan).dropna()

In [30]:
X = df_clear.iloc[:,:-1]
y = df_clear.iloc[:,-1]

clf_lg = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

print('LogisticRegression')
print('accuracy', cross_val_score(clf_lg, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_lg, X, y, cv=5, scoring = 'f1').mean())




clf_lsvc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LinearSVC())]
)
print('LinearSVC')
print('accuracy', cross_val_score(clf_lsvc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_lsvc, X, y, cv=5, scoring = 'f1').mean())


clf_svc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", SVC())]
)
print('SVC')
print('accuracy', cross_val_score(clf_svc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_svc, X, y, cv=5, scoring = 'f1').mean())


LogisticRegression
accuracy 0.8468445746108516
f1 0.6601244618889603
LinearSVC
accuracy 0.8485030154158197
f1 0.6616132960278641
SVC
accuracy 0.8356995943179577
f1 0.6267120053492474


Удаление признаков с '?' привело к незначительному снижению метрик. 

 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

In [31]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

clf_svc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", GradientBoostingClassifier())]
)
print('GradientBoostingClassifier')
print('accuracy', cross_val_score(clf_svc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_svc, X, y, cv=5, scoring = 'f1').mean())


clf_svc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier())]
)
print('RandomForestClassifier')
print('accuracy', cross_val_score(clf_svc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_svc, X, y, cv=5, scoring = 'f1').mean())


GradientBoostingClassifier
accuracy 0.8628987232494463
f1 0.6870302819762818
RandomForestClassifier
accuracy 0.848215568674882
f1 0.6690845154836111


GradientBoostingClassifier с наибольшей точностью и значением F1 меры классифицирует признаки.

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [32]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=10)

In [33]:
cols_train = X_train.columns

X_num_cols_train = X_train._get_numeric_data().columns
X_cat_cols_train = list(set(cols_train) - set(X_num_cols_train))
X_num_cols_train = X_num_cols_train.to_list()

In [34]:
ct_1 = ColumnTransformer([
    ('scale',  MinMaxScaler(), X_num_cols_train),
    ('onehot', OneHotEncoder(),  X_cat_cols_train)
])

ct_2 = ColumnTransformer([
    ('scale',  StandardScaler(), X_num_cols_train),
    ('onehot', OneHotEncoder(),  X_cat_cols_train)
])

pipe = Pipeline(steps=[
    ('fill_missing', SimpleImputer(missing_values='?', strategy='most_frequent')),
    ('prepare_columns', 'passthrough'),
    ('clf', 'passthrough')]
)

params = [
    {
        'prepare_columns': [ct_1, ct_2],
        'clf': [
            RandomForestClassifier(),
            GradientBoostingClassifier(),
            LogisticRegression(),
            LinearSVC()
        ]
    }
]

search = GridSearchCV(pipe, params, scoring=['accuracy', 'f1'], refit='f1', n_jobs=-1)
search.fit(X_train, y_train)
print(search.best_params_)



{'clf': RandomForestClassifier(), 'prepare_columns': ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                 [0, 2, 4, 10, 11, 12]),
                                ('onehot', OneHotEncoder(),
                                 [1, 3, 5, 6, 7, 8, 9, 13])])}


In [35]:
pred = search.predict(X_test)
print('accuracy',accuracy_score(y_test, pred)) 
print('f1_score',f1_score(y_test, pred)) 

accuracy 0.8483106315230851
f1_score 0.6747582021619571
